<a href="https://colab.research.google.com/github/atlas-github/Sandbox/blob/master/Text_Mining_with_R_Chapter_2_Sentiment_analysis_with_tidy_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Source: [Text Mining with R Chapter 2 Sentiment analysis with tidy data](https://www.tidytextmining.com/sentiment.html)